# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
matches = pd.read_csv('data/matches.csv', parse_dates=['date'])
deliveries = pd.read_csv('data/deliveries.csv')

In [ ]:
matches.head()

In [ ]:
deliveries.head()

#### Lets look at the shape of each dataframes

In [ ]:
print('There are {} rows and {} columns on deliveries dataframe.'.format(deliveries.shape[0], deliveries.shape[1]))
print('There are {} rows and {} columns on matches dataframe.'.format(matches.shape[0], matches.shape[1]))

#### 

# Data wrangling

There's a lot of missing values on umpire3, player_dismissed, dismissal_kind and fielder. It's normal to have no players being ejected from a match. And some matches don't have neccesarily need for the 3rd umpire. So we can leave the null values on those columns as is.

**To do:**

1. How to deal with missing values in city?
2. There are 372 columns with no winner and players of the match indicated, how to deal with this?
3. There are 500 rows with no umpire1 and umpire2. Given those, is it an official match? What can we do with this?

### Matches dataframe

#### Lets look at the matches dataframe.

In [ ]:
matches.isnull().sum()

#### Lets look at the rows with null in city column.

In [ ]:
matches.loc[matches['city'].isnull()]

We can replace city with NaN with Dubai since all of those NaN are just the matches that happened in Dubai International Cricket Stadium.

In [ ]:
matches.loc[matches.venue == 'Dubai International Cricket Stadium', 'city'] = 'Dubai'

All of the null values in city is the match that happenned in Dubai International Cricket Stadium.

#### Lets look at the rows with null in winner and player_of_match column.


In [ ]:
matches.loc[matches['winner'].isnull()]

In [ ]:
matches.loc[matches['player_of_match'].isnull()]

We can see that the null values on that columns are identical. Now notice that on the result column, rows are indicated to have no result. So I'm just going to delete those rows no.

In [ ]:
matches.loc[matches['result'] == 'no result']

#### Drop row with no result

In [ ]:
matches.drop(matches[matches['result'] == 'no result'].index, inplace=True)

### Deliveries dataframe

In [ ]:
deliveries.isnull().sum()

Since it is normal to have matches being evicted in the game.

In [ ]:
deliveries.loc[deliveries['fielder'].notna()]

#### Merge the two dataframes
I used inner join to merged the two dataframes.

In [ ]:
data = matches.merge(deliveries, left_on='id', right_on='match_id')

In [ ]:
data.head()

#### Lets look at the different data types and count the number null values in each column.

In [ ]:
# different datatypes in the merged dataframe
dataTypes = data.dtypes 

# count of null values in each columns
nullCnt = data.isnull().sum().sort_values(ascending=False) 

# concatenate
dataProperty = pd.concat([dataTypes, nullCnt], axis=1)

# rename columns
columns = ['data type', 'count of nulls']
dataProperty.columns = columns

# sort by count of nulls
dataProperty.sort_values(by='count of nulls', ascending=False, inplace=True)
dataProperty

Seems all the data types are good.

Let's look at the size of the resulting dataframe after merging.

In [ ]:
data.shape

The resulting dataframe has the same size that of the deliveries dataframe. I'm going to save the resulting dataframe into a csv file so I can later on do some analysis in PowerBI.

In [ ]:
data.to_csv('data/merged_data.csv')

## 1. What are the most successful teams?
I decided to label the most successful team with the highest number of wins.

In [ ]:
# count the win by each team
sucTeam = matches['winner'].value_counts().to_frame()

# rename column
sucTeam.rename(columns={'winner':'Count of Wins'}, inplace=True)
sucTeam.head()

In [ ]:
# visualize
sns.barplot(x=sucTeam.index, y='Count of Wins', data=sucTeam)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
print('The most successful team is {}.'.format(sucTeam.index[0]))

## Who are the most successful players? 
Count the how many times a player became the player of the match. The players with most number of times that become the player of the match are the most successful ones.

In [ ]:
# count how many times the player become the player of the match
playerOfMatch = matches['player_of_match'].value_counts().to_frame()

# rename column
playerOfMatch.rename(columns={'player_of_match':'Player of the Match Count'}, inplace=True)
playerOfMatch.head()

#### Select top 10 players

In [ ]:
topPlayers = playerOfMatch.iloc[:10]

In [ ]:
# visualize
sns.barplot(x=topPlayers.index, y='Player of the Match Count', data=topPlayers)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
print('The most successful player is {}.'.format(playerOfMatch.index[0]))

## What are the factors contributing win or loss of a team?

**To do:**

1. Does being in the homecourt have an advantage?
2. Does winning the toss coin has impact on the result of the match?
3. After winning the toss coin, does choosing to bat or the field has effect on the winning a match?

In [ ]:
batsmanGrp = deliveries.groupby(["match_id", "inning", "batting_team", "batsman"])
batsmen = batsmanGrp["batsman_runs"].sum().reset_index()
batsmen

## Suggest teams or players a company should endorse for its products.